# Problema de Otimização Não-Linear
## Projeto da disciplina **SME5720 - Otimização Não-linear**
### Estudo de caso da Regressão Linear

## Membros

* Eduardo Zaffari Monteiro - eduardozaffarimonteiro@usp.br - 12559490

* Gustavo Silva de Oliveira - gustavo.oliveira03@usp.br - 12567231

* Lucas Ivars Cadima Ciziks - luciziks@usp.br - 12559472

* Pedro Henrique de Freitas Maçonetto - pedromaconetto@usp.br - 12675419

## Introdução

Conforme definido por Montgomery (2021), *Regressão* é a metodologia responsável por estudar e compreender o **relacionamento estatístico** entre uma variável resposta (ou dependente) e variáveis preditoras (ou independentes). Esse método captura a correlação entre as variáveis em investigação e avalia se tal relação é estatisticamente significativa ou não. A técnica mais simples e comumente utilizada é a **Regressão Linear**, em que o relacionamento entre as variáveis é modelado de forma linear, procurando a reta ou plano que melhor se ajuste aos dados observados. O seguinte modelo com duas covariáveis ilustra a técnica:

$$
Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \varepsilon
$$

* $Y$: Variável Resposta ou Dependente;
* $X_1$ e $X_2$: Variáveis Preditoras ou Independente;
* $\beta_0$: Parâmetro que representa o intercepto;
* $\beta_1$ e $\beta_2$: Parâmetros cuja taxa de mudança quando as outras covariáveis são mantidas constate;
* $\varepsilon$: Erro Aleatório associado ao modelo; 

![Exemplo de Regressão](./Regressão.png "Exemplo de Regressao")

Desse modo, a regressão busca estimar os parâmetros do modelo proposto. Isso pode ser realizado de diversos modos, mas o método mais disseminado é o **Método dos Mínimos Quadrados** ou *Least-Squares*. Seu procedimento consiste em minimizar a soma dos erros ao quadrado, encontrando estimadores **não-viesados** para os parâmetros do modelo e com a **menor variância** dentre todos os estimadores não-viesados (Teorema de Gauss-Markov). Para o modelo de duas covariáveis, o problema é:

$$
\min \sum_{i=1}^{n} \varepsilon_i^2 = \sum_{i=1}^{n}(Y - \beta_0-\beta_1 X_1 - \beta_2 X_2)^2
$$

Essa otimização pode ser resolvida de forma analítica, todavia, o interesse aqui é em observar e avaliar o comportamento dos algoritmos de otimização não-linear vistos durante a disciplina no contexto da regressão linear. Assim, é possível comparar os resultados encontrados pelos métodos implementados com os resultados analíticos. 

Como o problema se trata de uma **minimização irrestrita**, os algoritmos a serem utilizados funcionam gerando uma sequência de iterandos xk a partir de um ponto inicial x0 . Essa iteração se encerra quando a solução é encontrada com precisão suficiente ou não há mais como progredir. Nesse contexto, há dois tipos de estratégias para se decidir como movimentar do ponto $x_k$ para $x_{k+1}$: **Busca Linear** e **Regiões de Confiança**.

Os algoritmos de busca linear calculam primeiramente a direção de procura, para assim encontrar, ao longo da direção, o tamanho do passo que minimiza a função de interesse e atinge o próximo $x_{k + 1}$. Já os métodos de região de confiança invertem a ordem: determinam a maior distância permitida entre os iterandos (isto é, o raio da região de confiança) e depois encontram a direção de iteração. Para resolver o problema de regressão, implementamos um método de busca linear, o Método de Newton, e um método de região de confiança, o **Método Dogleg**.


## Modelagem do Problema

O objetivo da regressão é estimar os $p$ parâmetros $\overrightarrow{{\beta}} = [\beta_0, \beta_1, ... \beta_{p-1}]$ que melhor ajustam o modelo aos dados observados. Nesse contexto, o método mais utilizado e poderoso é o **Mínimos Quadrados** ou **Least-Square**, cujo propósito é minimizar a soma quadrática dos erros ou ruídos gerados pelo modelo $S(\overrightarrow{{\beta}})$. Portanto, o problema da regressão é equivalente a resolver o seguinte processo de otimização não-linear:

$$
S(\overrightarrow{{\beta}}) = \sum_{i=1}^{n} \varepsilon_i^2 = \sum_{i=1}^{n}(Y - \beta_0-\beta_1 X_1 - ... - \beta_{p-1} X_{p-1})^2 = (\overrightarrow{Y} - \bm{X} \overrightarrow{{\beta}})^T (\overrightarrow{Y} - \bm{X} \overrightarrow{{\beta}})
$$

$$
\min{S(\overrightarrow{{\beta}})} = \min\sum_{i=1}^{n}(Y - \beta_0-\beta_1 X_1 - ... - \beta_{p-1} X_{p-1})^2
$$
com
* $\overrightarrow{{\beta}} \in \mathbb{R}^p$;
* $S \in \mathbb{R}^p \rightarrow \mathbb{R}$ é uma função suave. 

Como é possível observar, a função de interesse é quadrática, assim caracterizando um problema de natureza **não-linear** e **irrestrita**. Nessa conjuntura, tanto os métodos de busca linear quanto de região de confiança podem ser utilizados para resolvê-lo. No presente projeto, implementamos o **Método de Newton**, como representante dos algoritmos de busca linear, e um método de região de confiança, o **Método dogleg**. Assim, será possível comparar se os métodos atingiram o resultado correto e o número de iterações necessários para chegar em tal conclusão.

## Implementação

In [97]:
# Importando bibliotecas
import numpy as np
import pandas as pd
import scipy
from scipy import optimize
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.formula.api as smf

### Funções Auxiliares

Função a ser minimizada:
$$Y^TY -2Y^TX\beta  +  \beta^TX^TX\beta $$

In [98]:
# Função que transforma os dados do problema original em uma equação de Mínimos Quadrados
def f_least_square(betas, x, y):
  return y.T@y - 2*y.T@x@betas + betas.T@x.T@x@betas

Gradiente da função:
$$-2Y^TX  + X^TX\beta $$

In [99]:
# Definindo gradiente da função
def grad_f(betas, x, y):
  return -2*x.T@y + 2 * x.T@x@betas

Hessiana da função:
$$2 X^TX $$

In [100]:
# Definindo hessiana da função
def hess_f(betas, x, y):
  return (2)*(x.T@x)

Resolução analítica do alpha:
$$ \alpha_k = \frac{- \nabla f_k^T p_k}{p_k^T Q p_k} $$

In [101]:
# Definindo função para calculo de alpha
def alpha(betas, x, y, p):
    return (-grad_f(betas,x,y).T @ p)/(p.T @ hess_f(betas,x,y) @ p)

### Método de Newton

O método de Newton foi utilizado devido ao problema ser de natureza quadrática, neste caso específico, o método sempre encontra solução em uma iteração apenas, o que torna o método extremamente eficiente e torna desnecessário o cálculo do tamanho de passo '$a$', começaremos utilizando $a=1$, de forma a emular a solução do problema partindo de $1$ e reduzindo o tamanho do passo conforme necessário.
Na implementação, foi utilizada uma precisão de $10^{-4}$ para considerarmos o gradiente da função igual a $0$, ou seja, para acharmos um ponto estacionário e portanto uma solução do problema.

In [102]:
## Definindo método de newton
def newton_method(betasK, x, y):
  i = 0   # interacao
  while not(np.isclose(grad_f(betasK, x, y),np.zeros(len(grad_f(betasK, x, y))), atol=1e-04).all()):
    p = np.linalg.inv(hess_f(betasK, x, y)) @ grad_f(betasK, x, y)
    a = alpha(betasK,x,y,p)
    betasK = betasK + a * p
    i += 1
  return betasK, i

### Método dogleg

In [103]:
# Método dogleg
def dogleg(x0, x, y):
    minimium = scipy.optimize.minimize(f_least_square, x0=x0, jac=grad_f, hess=hess_f, args=(x,y), method="dogleg")
    return minimium

## Instâncias do Problema

### Produção de uma Fábrica

In [104]:
production_data = pd.read_csv("production.csv", index_col=0)
production_data.head()

,production,temperature,concentration
0,180,80,10
1,203,100,10
2,222,120,10
3,234,140,10
4,261,160,10


In [105]:
response_variable_prod = "production"
selected_regressors_prod = ["temperature", "concentration"]

y = production_data[response_variable_prod]
x = production_data[selected_regressors_prod].to_numpy().T
x0 = np.zeros(len(selected_regressors_prod) + 1)

In [106]:
# Aplicando Método de Newton
y = y.to_numpy()
ones = np.ones(20)
data_with_0 = np.vstack((ones,x))

newton_method(x0,data_with_0.T,y.T)

(array([88.14  ,  0.8925,  2.532 ]), 1)

In [107]:
# Aplicando Método dogleg
ones = np.ones(20)
data_with_0 = np.vstack((ones, x))

dogleg(x0, data_with_0.T, y.T)

     fun: 647.160000000149
    hess: array([[4.00e+01, 4.80e+03, 7.00e+02],
       [4.80e+03, 6.08e+05, 8.40e+04],
       [7.00e+02, 8.40e+04, 1.35e+04]])
     jac: array([ 0.00000000e+00, -2.32830644e-10,  2.91038305e-11])
 message: 'Optimization terminated successfully.'
    nfev: 8
    nhev: 7
     nit: 7
    njev: 8
  status: 0
 success: True
       x: array([88.14  ,  0.8925,  2.532 ])

## *Study on the Efficacy of Nosocomial Infection Control* (SENIC Data)

Nesse caso, utilizaremos dados de um estudo sobre a eficácia no controle de infecção nosocomial, isto é, infecções adquiridas durante a internação ou em procedimentos hospitalares. 

In [108]:
senic_data = pd.read_csv("./SENIC.csv")
senic_data.head()

,id,tempo,idade,risco_infeccao,cultura,x_ray,numero_camas,afiliacao,regiao,census,enfermeiras,servicos
0,1,7.13,55.7,4.1,9.0,39.6,279,2,4,207,241,60.0
1,2,8.82,58.2,1.6,3.8,51.7,80,2,2,51,52,40.0
2,3,8.34,56.9,2.7,8.1,74.0,107,2,3,82,54,20.0
3,4,8.95,53.7,5.6,18.9,122.8,147,2,4,53,148,40.0
4,5,11.20,56.5,5.7,34.5,88.9,180,2,1,134,151,40.0


Nesse problema de regressão linear múltipla, a **variável resposta** em estudo é o tempo que o paciente permanece no hospital. Desse modo, para entender o **tempo** de permanência, selecionaremos as seguintes covariáveis:
* Idade do paciente;
* Probabilidade do risco de infecção;
* Número de Enfermeiras;
* Cultura;
* Número de Camas oferecidas.



In [109]:
response_variable_senic = "tempo"
selected_regressors_senic = ["idade", "risco_infeccao", "cultura", "numero_camas", "enfermeiras"]

y = senic_data[response_variable_senic]
x = senic_data[selected_regressors_senic].to_numpy().T
x0 = np.zeros(len(selected_regressors_senic) + 1)

In [110]:
# Aplicando Método de Newton
y = y.to_numpy()
ones = np.ones(len(y))
x_with_0 = np.vstack((ones,x))

newton_method(x0,x_with_0.T,y.T)

(array([ 1.00647333,  0.09812137,  0.53168423,  0.02994805,  0.00617776,
        -0.00535391]),
 1)

In [111]:
# Aplicando Método dogleg
dogleg(x0, x_with_0.T, y.T)

     fun: 238.59959376881488
    hess: array([[2.2600000e+02, 1.2030400e+04, 9.8420000e+02, 3.5692000e+03,
        5.6990000e+04, 3.9154000e+04],
       [1.2030400e+04, 6.4485948e+05, 5.2392260e+04, 1.8768506e+05,
        3.0223468e+06, 2.0726958e+06],
       [9.8420000e+02, 5.2392260e+04, 4.6888200e+03, 1.7262320e+04,
        2.6902280e+05, 1.8699080e+05],
       [3.5692000e+03, 1.8768506e+05, 1.7262320e+04, 7.9831920e+04,
        9.6181180e+05, 6.8186000e+05],
       [5.6990000e+04, 3.0223468e+06, 2.6902280e+05, 9.6181180e+05,
        2.2701242e+07, 1.5380894e+07],
       [3.9154000e+04, 2.0726958e+06, 1.8699080e+05, 6.8186000e+05,
        1.5380894e+07, 1.1127790e+07]])
     jac: array([ 0.00000000e+00, -1.45519152e-11, -1.81898940e-12,  0.00000000e+00,
        1.16415322e-10, -5.82076609e-11])
 message: 'Optimization terminated successfully.'
    nfev: 3
    nhev: 2
     nit: 2
    njev: 3
  status: 0
 success: True
       x: array([ 1.00647333,  0.09812137,  0.53168423,  0.0299480

## Implementando a Solução Analítica para Fins de Comparação

### Produção de uma Fábrica

In [112]:
model = smf.ols(formula='production ~ temperature + concentration', data=production_data)
result = model.fit()
result.params

Intercept        88.1400
temperature       0.8925
concentration     2.5320
dtype: float64

## *Study on the Efficacy of Nosocomial Infection Control* (SENIC Data)

In [113]:
model = smf.ols(formula='tempo ~ idade + risco_infeccao + cultura + numero_camas + enfermeiras', data=senic_data)
result = model.fit()
result.params

Intercept         1.006473
idade             0.098121
risco_infeccao    0.531684
cultura           0.029948
numero_camas      0.006178
enfermeiras      -0.005354
dtype: float64

## Conclusão

Como previsto, para ambos os problemas, o método de Newton encontrou a solução em uma única iteração, enquanto o Dogleg precisou de mais iterações para atingir a resposta. Entretanto, mesmo havendo uma diferença no número de interações, e consequentemente no tempo de execução, os dois métodos convergiram para uma mesma solução em todos os casos.

Assim, é possível concluir que, para o problema de minimização do erro quadrático de uma regressão linear, o método de Newton é mais eficiente do que o método Dogleg, uma vez que ele encontra a solução em uma única iteração, o que era esperado já que se trata de um problema quadrático. Portanto, ambos os métodos foram eficazes e suficientes para estimar os parâmetros da regressão. 


## Referências

As seguintes referências foram utilizadas para o desenvolvimento do projeto:

* Material preparado e disponibilizado pela professora Marina Andretta.

* Neter, J., Kutner, M. H., Nachtsheim, C. J., & Wasserman, W. (1996). Applied linear statistical models.

* Montgomery, Douglas C., Elizabeth A. Peck, and G. Geoffrey Vining. Introduction to linear regression analysis. John Wiley & Sons, 2021.

* Chan, H. S. "Introduction to Probability for Data Science." (2021).